In [0]:
"""
Notebook d'harmonisation et de nettoyage des colonnes pour la préparation des données ML.

Ce notebook assure l'uniformité des datasets générés en harmonisant les colonnes entre tous
les fichiers train/test. Il supprime les colonnes inutiles, vérifie la cohérence et prépare
les données pour la modélisation en garantissant que tous les fichiers ont exactement les
mêmes features. Il effectue les opérations suivantes :

1. **Analyse des colonnes disponibles** :
   - Parcours du dossier datasets_par_pourcentage/
   - Inventaire des colonnes présentes dans chaque fichier CSV
   - Échantillonnage sur 5 premiers fichiers pour analyse rapide
   - Création d'un set all_columns_before avec toutes les colonnes uniques
   - Comptage et affichage du nombre total de colonnes trouvées
   - Identification des variations entre fichiers

2. **Définition des colonnes communes à conserver** :
   - Liste COLONNES_COMMUNES de 41 colonnes essentielles :
     * Métadonnées : Defaut, fichier, unit_id, pourcentage, pourcentage_reel
     * Variables temporelles : Nref, N
     * Températures (11) : T_amb, T_c_in/out, T_rec_*, T_cc_*, T_t_*
     * Pressions (11) : p_amb, p_c_in/out, p_rec_*, p_cc_*, p_t_*
     * Débits (12) : m_amb, m_c_in/out, m_rec_*, m_cc_*, m_t_*, m_fuel
   - Liste COLONNES_A_SUPPRIMER : Power_*, Eff_*, Beta_*, Is_Eff_*, etc.
   - Vérification de présence réelle dans les données
   - Rapport colonnes présentes vs absentes

3. **Nettoyage et harmonisation des fichiers** :
   - Boucle sur tous les fichiers CSV du dossier
   - Conservation uniquement des colonnes dans COLONNES_COMMUNES ET présentes
   - Suppression automatique des colonnes non désirées
   - Tracking des colonnes supprimées par fichier dans stats_nettoyage
   - Sauvegarde in-place des fichiers nettoyés
   - Gestion des erreurs avec try/except et logging
   - Progression affichée tous les 10 fichiers

4. **Vérification finale de cohérence** :
   - Rechargement d'un échantillon post-nettoyage
   - Vérification que tous les fichiers ont les mêmes colonnes
   - Calcul de la réduction : colonnes avant → après
   - Liste alphabétique des colonnes finales conservées
   - Détection des fichiers incohérents (normalement 0)
   - Validation "PARFAIT" si uniformité complète

5. **Analyse des features pour la modélisation** :
   - Catégorisation automatique par préfixe :
     * Températures : T_* (11 capteurs)
     * Pressions : p_* (11 capteurs)
     * Débits : m_* (12 capteurs)
     * Autres : Nref, N
   - Exclusion des métadonnées de l'analyse
   - Comptage par catégorie pour validation
   - Préparation pour la sélection de features

6. **Statistiques sur les données nettoyées** :
   - Analyse sur dataset_test_p50.csv comme échantillon représentatif
   - Comptage des features numériques utilisables
   - Calcul du taux de défaut dans l'échantillon
   - Vérification des valeurs manquantes (NaN)
   - Statistiques par colonne si valeurs manquantes détectées
   - Confirmation "Aucune valeur manquante" si données complètes

7. **Génération du rapport de nettoyage** :
   - Création rapport JSON avec métadonnées complètes
   - Statistiques globales : fichiers traités, colonnes avant/après
   - Liste des colonnes finales conservées
   - Décompte des features par catégorie
   - Exemples de colonnes supprimées (5 premiers)
   - Sauvegarde : rapport_nettoyage_colonnes.json
   - Timestamp d'exécution pour traçabilité

Paramètres et configuration :
- DOSSIER : /Volumes/dbe_dbx_internships/justin/predictive_maintenance/datasets_par_pourcentage
- Colonnes théoriques : 41 colonnes définies
- Colonnes à supprimer : 10 colonnes identifiées comme non pertinentes
- Format de sortie : CSV harmonisé, même structure pour tous

Résultats produits :
- Tous les fichiers CSV avec exactement les mêmes colonnes
- 0 valeur manquante (données complètes)

"""

In [0]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json

print("=" * 80)
print("MAINTENANCE PRÉDICTIVE - NETTOYAGE DES DONNÉES")
print("Notebook 3bis: Harmonisation des Colonnes entre Datasets")
print("=" * 80)
print(f"\nDate d'exécution: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n")

# === CONFIGURATION ===
DOSSIER = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/datasets_par_pourcentage"

print(" PHASE 1: ANALYSE DES COLONNES DISPONIBLES")
print("-" * 60)

# === Analyse initiale des colonnes ===
print(" Analyse des colonnes présentes dans chaque fichier...")

fichiers = [f for f in os.listdir(DOSSIER) if f.endswith(".csv") and not f.startswith("dataset_metadata")]
colonnes_par_fichier = {}
all_columns_before = set()

for fichier in fichiers[:5]:  # Échantillon pour analyse
    df = pd.read_csv(os.path.join(DOSSIER, fichier))
    colonnes_par_fichier[fichier] = list(df.columns)
    all_columns_before.update(df.columns)
    print(f"\n {fichier}:")
    print(f"   Nombre de colonnes: {len(df.columns)}")

print(f"\n✓ Total colonnes uniques trouvées: {len(all_columns_before)}")

# === Définition des colonnes à garder ===
print(f"\n\n PHASE 2: DÉFINITION DES COLONNES COMMUNES")
print("-" * 60)

# Colonnes communes RÉELLES (sans celles qui n'existent pas)
COLONNES_COMMUNES = [
    # Identifiants et métadonnées
    "Defaut", "fichier", "unit_id", "pourcentage", "pourcentage_reel",
    
    # Variables temporelles
     "Nref", "N",
    
    # Températures
    "T_amb", "T_c_in", "T_c_out", "T_rec_c_in", "T_rec_c_out", 
    "T_cc_in", "T_cc_out", "T_t_in", "T_t_out", "T_rec_h_in", "T_rec_h_out",
    
    # Pressions
    "p_amb", "p_c_in", "p_c_out", "p_rec_c_in", "p_rec_c_out",
    "p_cc_in", "p_cc_out", "p_t_in", "p_t_out", "p_rec_h_in", "p_rec_h_out",
    
    # Débits massiques
    "m_amb", "m_c_in", "m_c_out", "m_rec_c_in", "m_rec_c_out",
    "m_cc_in", "m_cc_out", "m_t_in", "m_t_out", "m_rec_h_in", "m_rec_h_out", "m_fuel"
]

# Colonnes à supprimer (qui peuvent exister dans certains fichiers)
COLONNES_A_SUPPRIMER = [
    "Power_Comp", "Is_Eff_Turb", "Is_Eff_Comp", "Power_Shaft", "Eff_Cycle",
    "Power_Turb", "Beta_Comp", "Beta_Turb", "Pdrop", "act_ash", "%"
]

print(f"✓ Colonnes à conserver: {len(COLONNES_COMMUNES)}")
print(f"✓ Colonnes à supprimer: {len(COLONNES_A_SUPPRIMER)}")

# Vérifier quelles colonnes définies sont réellement présentes
colonnes_presentes = []
colonnes_absentes = []

# Charger un fichier représentatif
df_sample = pd.read_csv(os.path.join(DOSSIER, fichiers[0]))
for col in COLONNES_COMMUNES:
    if col in df_sample.columns:
        colonnes_presentes.append(col)
    else:
        colonnes_absentes.append(col)

print(f"\n Statut des colonnes:")
print(f"   - Présentes: {len(colonnes_presentes)}")
print(f"   - Absentes: {len(colonnes_absentes)}")

if colonnes_absentes:
    print(f"\n  Colonnes absentes qui seront ignorées:")
    for col in colonnes_absentes:
        print(f"   - {col}")

# === Nettoyage des fichiers ===
print(f"\n\n PHASE 3: NETTOYAGE ET HARMONISATION")
print("-" * 60)

# Statistiques de nettoyage
stats_nettoyage = {
    'fichiers_traites': 0,
    'colonnes_supprimees': {},
    'erreurs': []
}

print(" Nettoyage en cours...")

for i, nom_fichier in enumerate(fichiers):
    try:
        chemin_fichier = os.path.join(DOSSIER, nom_fichier)
        df = pd.read_csv(chemin_fichier)
        
        colonnes_initiales = set(df.columns)
        
        # Ne garder que les colonnes qui existent ET sont dans COLONNES_COMMUNES
        cols_to_keep = [col for col in df.columns if col in colonnes_presentes]
        
        # Identifier les colonnes supprimées
        colonnes_supprimees = colonnes_initiales - set(cols_to_keep)
        if colonnes_supprimees:
            stats_nettoyage['colonnes_supprimees'][nom_fichier] = list(colonnes_supprimees)
        
        # Appliquer le nettoyage
        df_clean = df[cols_to_keep]
        
        # Sauvegarder
        df_clean.to_csv(chemin_fichier, index=False)
        stats_nettoyage['fichiers_traites'] += 1
        
        if (i + 1) % 10 == 0:
            print(f"    {i + 1}/{len(fichiers)} fichiers traités...")
            
    except Exception as e:
        stats_nettoyage['erreurs'].append({
            'fichier': nom_fichier,
            'erreur': str(e)
        })
        print(f"    Erreur sur {nom_fichier}: {e}")

print(f"\n Nettoyage terminé!")
print(f"   - Fichiers traités: {stats_nettoyage['fichiers_traites']}")
print(f"   - Fichiers avec erreurs: {len(stats_nettoyage['erreurs'])}")

# === Vérification finale ===
print(f"\n\n PHASE 4: VÉRIFICATION FINALE")
print("-" * 60)

# Vérifier la cohérence après nettoyage
all_columns_after = set()
sample_files = min(5, len(fichiers))

print(" Vérification de la cohérence...")
for fichier in fichiers[:sample_files]:
    df = pd.read_csv(os.path.join(DOSSIER, fichier))
    all_columns_after.update(df.columns)

print(f"\n Colonnes finales uniques: {len(all_columns_after)}")
print(f" Réduction: {len(all_columns_before)} → {len(all_columns_after)} colonnes")

# Afficher les colonnes finales
print(f"\n Liste des colonnes finales (ordre alphabétique):")
colonnes_finales = sorted(all_columns_after)
for i, col in enumerate(colonnes_finales, 1):
    print(f"   {i:2d}. {col}")

# Vérifier la cohérence parfaite
fichiers_incoherents = []
for fichier in fichiers:
    df = pd.read_csv(os.path.join(DOSSIER, fichier))
    if set(df.columns) != all_columns_after:
        fichiers_incoherents.append(fichier)

if fichiers_incoherents:
    print(f"\n  {len(fichiers_incoherents)} fichiers avec colonnes différentes détectés")
else:
    print(f"\n PARFAIT: Tous les fichiers ont exactement les mêmes colonnes!")

# === Analyse des features pour la modélisation ===
print(f"\n\n PHASE 5: ANALYSE DES FEATURES POUR LA MODÉLISATION")
print("-" * 60)

# Identifier les types de features
features_categories = {
    'Températures': [col for col in colonnes_finales if col.startswith('T_')],
    'Pressions': [col for col in colonnes_finales if col.startswith('p_')],
    'Débits': [col for col in colonnes_finales if col.startswith('m_')],
    'Autres': [col for col in colonnes_finales if not any(col.startswith(prefix) for prefix in ['T_', 'p_', 'm_']) 
               and col not in ['Defaut', 'fichier', 'unit_id', 'pourcentage', 'pourcentage_reel']]
}

print(" Répartition des features par catégorie:")
for categorie, features in features_categories.items():
    print(f"\n{categorie}: {len(features)} features")
    if len(features) <= 10:  # Afficher si pas trop nombreuses
        for feat in features:
            print(f"   - {feat}")

# === Statistiques sur un échantillon ===
print(f"\n\n PHASE 6: STATISTIQUES SUR LES DONNÉES NETTOYÉES")
print("-" * 60)

# Charger un dataset représentatif (p50)
df_p50 = pd.read_csv(os.path.join(DOSSIER, "dataset_test_p50.csv"))

# Features numériques seulement
features_numeriques = [col for col in df_p50.columns 
                      if col not in ['Defaut', 'fichier', 'unit_id', 'pourcentage', 'pourcentage_reel']]

print(f" Analyse sur dataset p50%:")
print(f"   - Échantillons: {len(df_p50)}")
print(f"   - Features numériques: {len(features_numeriques)}")
print(f"   - Taux de défaut: {df_p50['Defaut'].mean():.2%}")

# Vérifier les valeurs manquantes
missing_stats = df_p50[features_numeriques].isnull().sum()
if missing_stats.sum() > 0:
    print(f"\n  Valeurs manquantes détectées:")
    for col, count in missing_stats[missing_stats > 0].items():
        print(f"   - {col}: {count} ({count/len(df_p50)*100:.1f}%)")
else:
    print(f"\n Aucune valeur manquante!")

# === Rapport de nettoyage ===
print(f"\n\n GÉNÉRATION DU RAPPORT DE NETTOYAGE")
print("-" * 60)

rapport_nettoyage = {
    'date_execution': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'statistiques': {
        'fichiers_traites': stats_nettoyage['fichiers_traites'],
        'colonnes_avant': len(all_columns_before),
        'colonnes_apres': len(all_columns_after),
        'erreurs': len(stats_nettoyage['erreurs'])
    },
    'colonnes_finales': colonnes_finales,
    'features_par_categorie': {
        cat: len(feats) for cat, feats in features_categories.items()
    },
    'colonnes_supprimees_exemples': list(stats_nettoyage['colonnes_supprimees'].items())[:5]
}

# Sauvegarder le rapport
with open(os.path.join(DOSSIER, 'rapport_nettoyage_colonnes.json'), 'w') as f:
    json.dump(rapport_nettoyage, f, indent=4)

print(" Rapport sauvegardé: rapport_nettoyage_colonnes.json")

# === Résumé final ===
print(f"\n\n" + "=" * 80)
print("RÉSUMÉ DU NETTOYAGE")
print("=" * 80)

print(f"\n NETTOYAGE COMPLÉTÉ AVEC SUCCÈS:")
print(f"   - {stats_nettoyage['fichiers_traites']} fichiers harmonisés")
print(f"   - {len(all_columns_before) - len(all_columns_after)} colonnes supprimées")
print(f"   - {len(colonnes_finales)} colonnes conservées")

print(f"\n FEATURES DISPONIBLES POUR LA MODÉLISATION:")
print(f"   - {len(features_categories['Températures'])} capteurs de température")
print(f"   - {len(features_categories['Pressions'])} capteurs de pression") 
print(f"   - {len(features_categories['Débits'])} capteurs de débit")
print(f"   - {len(features_categories['Autres'])} autres features")

print(f"\n PROCHAINES ÉTAPES:")
print(f"   1. Normalisation des features (Notebook 4)")
print(f"   2. Sélection des features pertinentes (Notebook 5)")
print(f"   3. Modélisation et optimisation (Notebooks 6-8)")

print(f"\n REMARQUE IMPORTANTE:")
print(f"   Les colonnes théoriques suivantes n'étaient pas présentes dans les données:")
print(f"   - Power_Shaft, Eff_Cycle, Power_Comp, Power_Turb")
print(f"   - Beta_Comp, Beta_Turb, Is_Eff_Comp, Is_Eff_Turb")
print(f"   → Ces features ne seront pas utilisées dans la modélisation")

print("\n" + "=" * 80)
print("FIN DU NOTEBOOK 3bis - DONNÉES NETTOYÉES ET HARMONISÉES")
print("=" * 80)

MAINTENANCE PRÉDICTIVE - NETTOYAGE DES DONNÉES
Notebook 3bis: Harmonisation des Colonnes entre Datasets

Date d'exécution: 2025-06-18 08:55:26


 PHASE 1: ANALYSE DES COLONNES DISPONIBLES
------------------------------------------------------------
 Analyse des colonnes présentes dans chaque fichier...

 dataset_test_p0.csv:
   Nombre de colonnes: 105

 dataset_test_p1.csv:
   Nombre de colonnes: 105

 dataset_test_p10.csv:
   Nombre de colonnes: 105

 dataset_test_p100.csv:
   Nombre de colonnes: 105

 dataset_test_p2.csv:
   Nombre de colonnes: 105

✓ Total colonnes uniques trouvées: 105


 PHASE 2: DÉFINITION DES COLONNES COMMUNES
------------------------------------------------------------
✓ Colonnes à conserver: 41
✓ Colonnes à supprimer: 11

 Statut des colonnes:
   - Présentes: 39
   - Absentes: 2

  Colonnes absentes qui seront ignorées:
   - unit_id
   - pourcentage_reel


 PHASE 3: NETTOYAGE ET HARMONISATION
------------------------------------------------------------
 Nettoy